In [1]:
import pandas as pd
import json

In [2]:
with open("data/data_summoner_match_data_야식은치킨이지.json", 'r') as file:
    raw_data = json.load(file)

In [3]:
df = pd.json_normalize(raw_data)
df_meta = df.iloc[:, 1:3]
df_meta.columns = ['matchId', 'participants']
df_info = df.iloc[:, 3:]
df_info.columns = raw_data[0]['info'].keys()

In [20]:
arr_teams = df_info.pop('teams')

In [24]:
arr_participants = df_info.pop('participants')

In [6]:
df_teams = pd.concat([pd.json_normalize(arr_teams[x]) for x in range(len(raw_data))])
df_teams.index = range(len(raw_data)*2)

In [25]:
df_participants = pd.concat([pd.json_normalize(arr_participants[x]) for x in range(len(raw_data))])
df_participants.index = range(len(raw_data)*10)

## now we have 
- df_meta
- df_info
- df_teams
- df_participants

In [27]:
print(
    df_meta.columns, 
    df_info.columns,
    df_teams.columns,
    df_participants.columns,
    sep='\n')

Index(['matchId', 'participants'], dtype='object')
Index(['gameCreation', 'gameDuration', 'gameId', 'gameMode', 'gameName',
       'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId', 'platformId',
       'queueId', 'tournamentCode'],
      dtype='object')
Index(['bans', 'teamId', 'win', 'objectives.baron.first',
       'objectives.baron.kills', 'objectives.champion.first',
       'objectives.champion.kills', 'objectives.dragon.first',
       'objectives.dragon.kills', 'objectives.inhibitor.first',
       'objectives.inhibitor.kills', 'objectives.riftHerald.first',
       'objectives.riftHerald.kills', 'objectives.tower.first',
       'objectives.tower.kills'],
      dtype='object')
Index(['assists', 'baronKills', 'bountyLevel', 'champExperience', 'champLevel',
       'championId', 'championName', 'championTransform',
       'consumablesPurchased', 'damageDealtToBuildings',
       ...
       'unrealKills', 'visionScore', 'visionWardsBoughtInGame', 'wardsKilled',
       'wardsPlace

In [44]:
with open("data/data_summoner_야식은치킨이지.json", 'r') as file:
    summoner_puuid = json.load(file)["puuid"]
summoner_puuid

'mX__RNvnhclbNgyGz8tK9-TcnJRwHeTk0FyHL2jOJe6MU6WKNvwCkdfaaLJaZzujUoz5YACjf6-p_g'

In [49]:
df_meta['summonerIndex'] = df_meta.participants.map(lambda participants : participants.index(summoner_puuid))
df_meta.head()

,matchId,participants,summonerIndex
0,KR_5383029593,[rUTte1LB7uqx7-Be9GDunKU58Cvda2Hl3zslnmrD35alP...,6
1,KR_5383081525,[FMr_19OZMaYzwkkUXY2ILgKlspRcKpW_m3Ich7aZQNmCO...,5
2,KR_5382976870,[mX__RNvnhclbNgyGz8tK9-TcnJRwHeTk0FyHL2jOJe6MU...,0
3,KR_5382817808,[R9gC34TdorS8Ry23GXmm46K5jLi6-hCary5cX3XA4Oko7...,3
4,KR_5382882767,[Y97ddFWUFhOEUA0ULYUc-jlBVe_sQROMuOVQQdCwCLREI...,5


# 승률

In [121]:
# 나는 어떤 팀이었을까
is_redteam = [int(x >= 5) for x in df_meta.summonerIndex]
index = [x*2+is_redteam[x] for x in range(len(raw_data))]

[1,
 3,
 4,
 6,
 9,
 10,
 12,
 15,
 16,
 19,
 20,
 22,
 24,
 27,
 29,
 30,
 32,
 34,
 36,
 39,
 40,
 43,
 45,
 47,
 48,
 50,
 52,
 55,
 56,
 59,
 60,
 63,
 65,
 67,
 68,
 71,
 72,
 75,
 77,
 79,
 80,
 82,
 84,
 86,
 89,
 90,
 93,
 95,
 96,
 99,
 101,
 103,
 105,
 107,
 109,
 111,
 112,
 114,
 117,
 119,
 121,
 122,
 125,
 127,
 129,
 130,
 132,
 134,
 136,
 138,
 140,
 142,
 144,
 146,
 148,
 151,
 153,
 154,
 157,
 158,
 161,
 163,
 165,
 166,
 169,
 171,
 173,
 175,
 176,
 179,
 181,
 183,
 185,
 186,
 188,
 191,
 192,
 195,
 197,
 198]

In [74]:
win_sum = sum(df_teams.win[index])
win_rate = win_sum / len(raw_data)
print(win_sum, win_rate)

53 0.53


# 레드팀 / 블루팀 각 횟수와 승률

In [98]:
df_blueteam = df_teams.iloc[index]
df_blueteam

,bans,teamId,win,objectives.baron.first,objectives.baron.kills,objectives.champion.first,objectives.champion.kills,objectives.dragon.first,objectives.dragon.kills,objectives.inhibitor.first,objectives.inhibitor.kills,objectives.riftHerald.first,objectives.riftHerald.kills,objectives.tower.first,objectives.tower.kills
1,[],200,False,False,0,False,27,False,0,False,0,False,0,False,0
3,[],200,False,False,0,True,24,False,1,False,0,False,0,False,3
4,[],100,False,False,0,False,15,True,1,False,0,False,0,False,0
6,[],100,False,False,0,True,28,True,1,False,0,True,2,True,6
9,[],200,False,False,0,False,7,False,0,False,0,True,1,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,[],200,True,False,0,False,45,False,0,True,2,False,0,True,4
192,[],100,False,False,1,False,54,False,1,False,0,True,1,True,7
195,[],200,True,False,0,False,46,False,0,True,2,False,0,False,4
197,[],200,True,True,2,True,61,False,4,False,1,False,0,True,7


In [ ]:
win_sum_blue = sum(df_teams[df_teams.teamId==100].win[index])
win_rate_blue = win_sum_blue / len(raw_data)
print(win_sum, win_rate)